Modelo 1: Predicción a partir de datos de la base de datos.

In [20]:
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl
import re

In [21]:
# Cargamos el dataset, el cual está comprimido en un .zip
ruta_zip = '../data/RepoRT_classified_CCinformation.zip'
nombre_tsv = 'RepoRT_classified_CCinformation.tsv'

with zipfile.ZipFile(ruta_zip, 'r') as archivo_zip:
    with archivo_zip.open(nombre_tsv) as archivo_tsv:
        dataset = pd.read_csv(archivo_tsv, sep='\t')


C:\Users\cmora\AppData\Local\Temp\ipykernel_24868\496839114.py:7: DtypeWarning: Columns (1,13) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(archivo_tsv, sep='\t')


In [23]:
# Ponemos todos los datos de las columnas del mismo tipo
dataset['name'] = dataset['name'].astype('str')
dataset['comment'] = dataset['comment'].astype('str')

Estructura de los datos

In [24]:
class Column:
    def __init__(self, name, usp_code, length, id, particle_size, temperature, flowrate, t0):
        self.name = name
        self.usp_code = usp_code
        self.length = length
        self.id = id
        self.particle_size = particle_size
        self.temperature = temperature
        self.flowrate = flowrate
        self.t0 = t0
    
    def __eq__(self, value):
        if not isinstance(value, Column):
            return False
        return (
            self.name == value.name and
            self.usp_code == value.usp_code and
            self.length == value.length and
            self.id == value.id and
            self.particle_size == value.particle_size and
            self.temperature == value.temperature and
            self.flowrate == value.flowrate and
            self.t0 == value.t0
        )
    
    def __hash__(self):
        return hash((
            self.name,
            self.usp_code,
            self.length,
            self.id,
            self.particle_size,
            self.temperature,
            self.flowrate,
            self.t0
        ))
    
    def __repr__(self):
        return f"Config(eluyente1={self.eluyente1}, eluyente2={self.eluyente2}, columna={repr(self.columna)})"
    def __str__(self):
        return f"Config(eluyente1={self.eluyente1}, eluyente2={self.eluyente2}, columna={repr(self.columna)})"

In [25]:
class Config:
    def __init__(self, eluyente1, eluyente2, columna:Column):
        self.eluyente1 = eluyente1
        self.eluyente2 = eluyente2
        self.columna = columna

    def __eq__(self, value):
        if not isinstance(value, Config):
            return False
        return (
            self.eluyente1 == value.eluyente1 and
            self.eluyente2 == value.eluyente2 and
            self.columna == value.columna
        )
    
    def __hash__(self):
        return hash((self.eluyente1, self.eluyente2, self.columna))
    
    def __repr__(self):
        return (f"Config(eluyente1={self.eluyente1}, eluyente2={self.eluyente2}, "
                    f"columna=Column(name={self.columna.name}, usp_code={self.columna.usp_code}, "
                    f"length={self.columna.length}, id={self.columna.id}, "
                    f"particle_size={self.columna.particle_size}, temperature={self.columna.temperature}, "
                    f"flowrate={self.columna.flowrate}, t0={self.columna.t0}))")
    
    def __str__(self):
            # Accediendo a los atributos de la columna desde el objeto columna
            return (f"Config(eluyente1={self.eluyente1}, eluyente2={self.eluyente2}, "
                    f"columna=Column(name={self.columna.name}, usp_code={self.columna.usp_code}, "
                    f"length={self.columna.length}, id={self.columna.id}, "
                    f"particle_size={self.columna.particle_size}, temperature={self.columna.temperature}, "
                    f"flowrate={self.columna.flowrate}, t0={self.columna.t0}))")

Filtramos el dataset y nos quedamos con los datos que nos importan

In [26]:
eluent_columns = [
    f"eluent.{i}.{compound} 0"
    for i in [1, 2]  # Para eluent.1 y eluent.2
    for compound in [
        "h2o", "meoh", "acn", "iproh", "acetone", "hex", "chcl3", "ch2cl2", "hept",
        "formic", "acetic", "trifluoroacetic", "phosphor", "nh4ac", "nh4form",
        "nh4carb", "nh4bicarb", "nh4f", "nh4oh", "trieth", "triprop", "tribut",
        "nndimethylhex", "medronic", "pH"
    ]
]

# Lista original de columnas
columns_to_extract = [
    "id",
    "column.name", "column.usp.code_0", "column.usp.code_L1", "column.usp.code_L10",
    "column.usp.code_L109", "column.usp.code_L11", "column.usp.code_L114", "column.usp.code_L122",
    "column.usp.code_L3", "classyfire.class", "rt", "alternative_parents",
    "column.usp.code_L43", "column.usp.code_L68", "column.usp.code_L7", "column.length",
    "column.id", "column.particle.size", "column.temperature", "column.flowrate", "column.t0"
]

# Generar nombres de columnas para "t 0", ..., "t 17"
t_columns = [f"t {i}" for i in range(18)]

# Combinar todas las listas
columns_to_extract += eluent_columns + t_columns

data = dataset[columns_to_extract]
print(data.columns)

Index(['id', 'column.name', 'column.usp.code_0', 'column.usp.code_L1',
       'column.usp.code_L10', 'column.usp.code_L109', 'column.usp.code_L11',
       'column.usp.code_L114', 'column.usp.code_L122', 'column.usp.code_L3',
       'classyfire.class', 'rt', 'alternative_parents', 'column.usp.code_L43',
       'column.usp.code_L68', 'column.usp.code_L7', 'column.length',
       'column.id', 'column.particle.size', 'column.temperature',
       'column.flowrate', 'column.t0', 'eluent.1.h2o 0', 'eluent.1.meoh 0',
       'eluent.1.acn 0', 'eluent.1.iproh 0', 'eluent.1.acetone 0',
       'eluent.1.hex 0', 'eluent.1.chcl3 0', 'eluent.1.ch2cl2 0',
       'eluent.1.hept 0', 'eluent.1.formic 0', 'eluent.1.acetic 0',
       'eluent.1.trifluoroacetic 0', 'eluent.1.phosphor 0', 'eluent.1.nh4ac 0',
       'eluent.1.nh4form 0', 'eluent.1.nh4carb 0', 'eluent.1.nh4bicarb 0',
       'eluent.1.nh4f 0', 'eluent.1.nh4oh 0', 'eluent.1.trieth 0',
       'eluent.1.triprop 0', 'eluent.1.tribut 0', 'eluent.1.nn

In [27]:
def create_config_objects(dataset):
    def get_uspcode(row):
        usp_columns = [
            'column.usp.code_0', 'column.usp.code_L1', 'column.usp.code_L10',
            'column.usp.code_L109', 'column.usp.code_L11', 'column.usp.code_L114',
            'column.usp.code_L122', 'column.usp.code_L3', 'column.usp.code_L43',
            'column.usp.code_L68', 'column.usp.code_L7'
        ]
        # Busca la primera columna con valor 1 y devuelve su nombre
        return next((col for col in usp_columns if row[col] == 1), None)

    config_objects = set()

    for _, row in dataset.iterrows():
        # Crear el objeto Column
        columna = Column(
            name=row['column.name'],
            usp_code=get_uspcode(row),  # Llama a la función para determinar el USP Code
            length=row['column.length'],
            id=row['column.id'],
            particle_size=row['column.particle.size'],
            temperature=row['column.temperature'],
            flowrate=row['column.flowrate'],
            t0=row['column.t0']
        )
        
        # Crear el objeto Config
        
        eluyente1 = next((key for key in dataset.columns if key.startswith('eluent.1.') and row[key] == 100), None)
        eluyente2 = next((key for key in dataset.columns if key.startswith('eluent.2.') and row[key] == 100), None)

        config = Config(eluyente1=eluyente1, eluyente2=eluyente2, columna=columna)
        config_objects.add(config)

    return list(config_objects)


In [28]:
config_objects = create_config_objects(data)

print(len(config_objects))


136


De 164346 registros de configuraciones hay 136 que son distintas tomando en cuenta que definimos configuración como:

* Eluyentes utilizados (eluyente 1 y eluyente 2).

* Columna cromatográfica (Todos los parámetros de la columna cromatográfica son los mismos).

Funciones de scoring

Factor de selectividad.

$$
\alpha = \frac{t_{R,A} - t_0}{t_{R,B} - t_0}
$$

* RTa: Tiempo de retención del metabolito A.
* RTb: Tiempo de retención del metabolito B.
* 𝑡0: Tiempo de retención del soluto no retenido (tiempo muerto).
$$
\text{Tiempo muerto} = \frac{\text{longitud de la columna}}{\text{Flowrate}}
$$

In [29]:
def alpha(rta, rtb, t0):
    return (rta - t0) / (rtb - t0)

In [31]:

def t0(length, flowrate):
    return length/flowrate

In [30]:
def filter_by_config(data_family, config):
    
    usp_columns = [
        'column.usp.code_0', 'column.usp.code_L1', 'column.usp.code_L10',
        'column.usp.code_L109', 'column.usp.code_L11', 'column.usp.code_L114',
        'column.usp.code_L122', 'column.usp.code_L3', 'column.usp.code_L43',
        'column.usp.code_L68', 'column.usp.code_L7'
    ]
    # Crear una lista de condiciones para las columnas usp_code
    usp_conditions = [
        (data_family[usp_col] == 1) if usp_col == config.columna.usp_code else (data_family[usp_col] == 0)
        for usp_col in usp_columns
    ]

    # Combinar todas las condiciones usando '&'
    combined_usp_condition = usp_conditions[0]
    for condition in usp_conditions[1:]:
        combined_usp_condition &= condition

    # Crear la condición general
    combined_condition = (
        (data_family['column.name'] == config.columna.name) &
        (data_family['column.length'] == config.columna.length) &
        (data_family['column.id'] == config.columna.id) &
        (data_family['column.particle.size'] == config.columna.particle_size) &
        (data_family['column.temperature'] == config.columna.temperature) &
        (data_family['column.flowrate'] == config.columna.flowrate) &
        (data_family['column.t0'] == config.columna.t0) &
        combined_usp_condition  # Condiciones combinadas de usp_code
    )

    # Filtrar los datos que cumplen la condición
    return data_family[combined_condition]


In [16]:
def main(data, family, fscore):
    # Separar los datasets por experimento
    data['experiment'] = data['id'].apply(lambda x: x.split('_')[0])  # Extraer el experimento del 'id'
    experiments = data['experiment'].unique()  # Obtener todos los experimentos únicos
    
    result_datasets = []  # Lista para almacenar los datasets filtrados por configuración
    
    # Eliminar texto dentro de paréntesis y guardarlo en una variable
    match = re.search(r'\((.*?)\)', family)  # Buscar texto entre paréntesis
    if match:
        family_name = family.split('(')[0].strip()  # Eliminar el texto entre paréntesis
        family_in_parentheses = match.group(1)  # Lo que está dentro de los paréntesis
    else:
        family_name = family
        family_in_parentheses = None

    # Para cada experimento
    for exp in experiments:
        print(f"Procesando experimento: {exp}")
        
        # Filtrar datos por el experimento
        experiment_data = data[data['experiment'] == exp]
        
        # Obtención de un conjunto con los posibles padres
        alternative_parents = [set(item.split(",")) for item in experiment_data["alternative_parents"]]
        class_attr = experiment_data["classyfire.class"]
        families = [parent_set.union({cls}) for parent_set, cls in zip(alternative_parents, class_attr)]
        
        # Filtrar por familia (usando `family_name` y `family`)
        data_family = experiment_data[[family_name in conjunto or family in conjunto for conjunto in families]]
        
        print(f"Datos filtrados por familia en el experimento {exp}:")
        print(data_family)

        # Crear configuraciones usando `fscore` si es necesario
        configs = create_config_objects(data_family)
        print(f"Cantidad de configuraciones: {len(configs)}")
        
        # Para todos los datos de una misma configuración
        datasets_by_config = [
            filter_by_config(data_family, config) 
            for config in configs
        ]
        
        print(f"Cantidad de datasets generados para el experimento {exp}: {len(datasets_by_config)}")
        
        # Agregar los datasets generados a la lista final
        result_datasets.extend(datasets_by_config)

    # Ordenar por RT en una lista
    for i in range(len(datasets_by_config)):
        datasets_by_config[i] = datasets_by_config[i].sort_values(by="rt")

    
    resultados = []
    # Iteramos sobre los datasets ya ordenados
    for dataset in datasets_by_config:
        resultados_dataset = []  # Almacena resultados para el dataset actual

        # Calcular t0 con la primera fila del dataset
        primer_elemento = dataset.iloc[0]
        t0_value = t0(primer_elemento['column.length'], primer_elemento['column.flowrate'])
        
        # Iterar en pares desde la segunda fila
        for i in range(1, len(dataset)):
            elemento_n_menos_1 = dataset.iloc[i - 1]
            elemento_n = dataset.iloc[i]
            
            # Aplicar alpha usando rt de los elementos y t0
            resultado = alpha(
                elemento_n_menos_1['rt'],
                elemento_n['rt'],
                t0_value
            )
            
            resultados_dataset.append(resultado)
        
        resultados.append(resultados_dataset)

    lista_tuplas = []
    for i in range(len(resultados)):
        lista_tuplas.append((configs[i], fscore(resultados[i])))

    return lista_tuplas



In [17]:
import re

def main(data, family, fscore):
    # Separar los datasets por experimento
    data['experiment'] = data['id'].apply(lambda x: x.split('_')[0])  # Extraer el experimento del 'id'
    experiments = data['experiment'].unique()  # Obtener todos los experimentos únicos

    result_datasets = []  # Lista para almacenar los datasets filtrados por configuración

    # Eliminar texto dentro de paréntesis y guardarlo en una variable
    match = re.search(r'\((.*?)\)', family)  # Buscar texto entre paréntesis
    if match:
        family_name = family.split('(')[0].strip()  # Eliminar el texto entre paréntesis
        family_in_parentheses = match.group(1)  # Lo que está dentro de los paréntesis
    else:
        family_name = family
        family_in_parentheses = None

    # Para cada experimento
    for exp in experiments:
        # Filtrar datos por el experimento
        experiment_data = data[data['experiment'] == exp]

        # Obtención de un conjunto con los posibles padres
        alternative_parents = [set(item.split(",")) for item in experiment_data["alternative_parents"]]
        class_attr = experiment_data["classyfire.class"]
        families = [parent_set.union({cls}) for parent_set, cls in zip(alternative_parents, class_attr)]

        # Filtrar por familia (usando `family_name` y `family`)
        data_family = experiment_data[[family_name in conjunto or family in conjunto for conjunto in families]]

        # Crear configuraciones usando `fscore` si es necesario
        configs = create_config_objects(data_family)

        # Para todos los datos de una misma configuración
        datasets_by_config = [
            filter_by_config(data_family, config) 
            for config in configs
        ]

        # Agregar los datasets generados a la lista final
        result_datasets.extend(datasets_by_config)

    # Ordenar los datasets por RT
    for i in range(len(result_datasets)):
        result_datasets[i] = result_datasets[i].sort_values(by="rt")

    resultados = []
    # Iteramos sobre los datasets ya ordenados
    for dataset in result_datasets:
        if dataset.empty:
            continue  # Saltar datasets vacíos
        
        resultados_dataset = []  # Almacena resultados para el dataset actual

        # Calcular t0 con la primera fila del dataset
        primer_elemento = dataset.iloc[0]
        t0_value = t0(primer_elemento['column.length'], primer_elemento['column.flowrate'])

        # Iterar en pares desde la segunda fila
        for i in range(1, len(dataset)):
            elemento_n_menos_1 = dataset.iloc[i - 1]
            elemento_n = dataset.iloc[i]

            # Aplicar alpha usando rt de los elementos y t0
            resultado = alpha(
                elemento_n_menos_1['rt'],
                elemento_n['rt'],
                t0_value
            )

            resultados_dataset.append(resultado)

        resultados.append(resultados_dataset)

    # Crear lista de tuplas con verificación de longitudes
    lista_tuplas = []
    for i in range(len(result_datasets)):
        if i < len(resultados):
            lista_tuplas.append((configs[i], fscore(resultados[i])))
        else:
            print(f"Advertencia: No hay suficientes resultados para config {configs[i]}")

    return lista_tuplas


In [35]:
def fscore_peor_caso(lista:list):
    if not lista:
        return 0
    return min(lista)

def fscore_caso_medio(lista:list):
    if not lista:
        return 0
    return sum(lista)/len(lista)

def fscore_mejor_caso(lista:list):
    if not lista:
        return 0
    return max(lista)


In [19]:
resultado = main(data, "Flavonoids (CHEMONTID:0000334)", fscore=fscore_peor_caso)

C:\Users\cmora\AppData\Local\Temp\ipykernel_24868\4131909165.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['experiment'] = data['id'].apply(lambda x: x.split('_')[0])  # Extraer el experimento del 'id'


IndexError: list index out of range

In [ ]:
print(resultado)

[(<__main__.Config object at 0x00000167AB2C4F80>, np.float64(1.0))]


In [ ]:
def dataset_by_config_with_experiment(data, family, fscore=None):
    # Separar los datasets por experimento
    data['experiment'] = data['id'].apply(lambda x: x.split('_')[0])  # Extraer el experimento del 'id'
    experiments = data['experiment'].unique()  # Obtener todos los experimentos únicos
    
    result_datasets = []  # Lista para almacenar los datasets filtrados por configuración
    
    # Eliminar texto dentro de paréntesis y guardarlo en una variable
    match = re.search(r'\((.*?)\)', family)  # Buscar texto entre paréntesis
    if match:
        family_name = family.split('(')[0].strip()  # Eliminar el texto entre paréntesis
        family_in_parentheses = match.group(1)  # Lo que está dentro de los paréntesis
    else:
        family_name = family
        family_in_parentheses = None

    # Para cada experimento
    for exp in experiments:
        print(f"Procesando experimento: {exp}")
        
        # Filtrar datos por el experimento
        experiment_data = data[data['experiment'] == exp]
        
        # Obtención de un conjunto con los posibles padres
        alternative_parents = [set(item.split(",")) for item in experiment_data["alternative_parents"]]
        class_attr = experiment_data["classyfire.class"]
        families = [parent_set.union({cls}) for parent_set, cls in zip(alternative_parents, class_attr)]
        
        # Filtrar por familia (usando `family_name` y `family`)
        data_family = experiment_data[[family_name in conjunto or family in conjunto for conjunto in families]]
        
        print(f"Datos filtrados por familia en el experimento {exp}:")
        print(data_family)

        # Crear configuraciones usando `fscore` si es necesario
        configs = create_config_objects(data_family)
        print(f"Cantidad de configuraciones: {len(configs)}")
        
        # Para todos los datos de una misma configuración
        datasets_by_config = [
            filter_by_config(data_family, config) 
            for config in configs
        ]
        
        print(f"Cantidad de datasets generados para el experimento {exp}: {len(datasets_by_config)}")
        
        # Agregar los datasets generados a la lista final
        result_datasets.extend(datasets_by_config)
    
    return result_datasets


In [ ]:
list_datasets_filtered = dataset_by_config_with_experiment(data, "Flavonoids (CHEMONTID:0000334)")

C:\Users\cmora\AppData\Local\Temp\ipykernel_17744\2582709829.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['experiment'] = data['id'].apply(lambda x: x.split('_')[0])  # Extraer el experimento del 'id'


Procesando experimento: 0001
Datos filtrados por familia en el experimento 0001:
            id                 column.name  column.usp.code_0  \
0   0001_00001  Waters ACQUITY UPLC HSS T3                0.0   
1   0001_00002  Waters ACQUITY UPLC HSS T3                0.0   
2   0001_00003  Waters ACQUITY UPLC HSS T3                0.0   
10  0001_00011  Waters ACQUITY UPLC HSS T3                0.0   
14  0001_00015  Waters ACQUITY UPLC HSS T3                0.0   
15  0001_00016  Waters ACQUITY UPLC HSS T3                0.0   
16  0001_00017  Waters ACQUITY UPLC HSS T3                0.0   
18  0001_00019  Waters ACQUITY UPLC HSS T3                0.0   
19  0001_00020  Waters ACQUITY UPLC HSS T3                0.0   
20  0001_00021  Waters ACQUITY UPLC HSS T3                0.0   
22  0001_00023  Waters ACQUITY UPLC HSS T3                0.0   
23  0001_00024  Waters ACQUITY UPLC HSS T3                0.0   
24  0001_00025  Waters ACQUITY UPLC HSS T3                0.0   
28  0001_

In [ ]:
def dataset_by_config(data, family, fscore = None):
    
    # Eliminar texto dentro de paréntesis y guardarlo en una variable
    match = re.search(r'\((.*?)\)', family)  # Busca texto entre paréntesis
    if match:
        family_name = family.split('(')[0].strip()  # Elimina el texto entre paréntesis
        family_in_parentheses = match.group(1)  # Lo que está dentro de los paréntesis
    else:
        family_name = family
        family_in_parentheses = None

    # Obtención de un conjunto con los posibles padres
    alternative_parents = [set(item.split(",")) for item in data["alternative_parents"]]
    class_attr = data["classyfire.class"]
    families = [parent_set.union({cls}) for parent_set, cls in zip(alternative_parents, class_attr)]

    # Si está el nombre con parentesis o sin parentesis <- Esto es por que en alternative_parents lo ponen sin parentesis.
    data_family = data[[family_name in conjunto or family in conjunto for conjunto in families]]

    print(data_family)

    configs = create_config_objects(data_family)

    print(len(configs))

    # Para todos los datos de una misma configuración
    datasets_by_config = [
        filter_by_config(data_family, config)
        for config in configs
    ]
    
    print(f"Cantidad de datasets generados: {len(datasets_by_config)}")

    return datasets_by_config

Crear un excel con los datasets

In [ ]:
def save_excel(datasets_by_config, output_file):
    #datasets_by_config = dataset_by_config(data, "Flavonoids (CHEMONTID:0000334)")
    dataset_names = [f"Dataset_{i+1}" for i in range(len(datasets_by_config))]
    # Ruta donde se guardará el archivo Excel
    #output_file = "../data/datasets_by_config_2.xlsx"

    # Crear un archivo Excel con una hoja por cada dataset
    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        for df, name in zip(datasets_by_config, dataset_names):
            df.to_excel(writer, sheet_name=name[:31], index=False)  # El nombre de hoja tiene un límite de 31 caracteres

    print(f"Archivo Excel creado: {output_file}")


In [ ]:
save_excel(list_datasets_filtered, "../data/datasets_by_config_2_0.xlsx")

Archivo Excel creado: ../data/datasets_by_config_2_0.xlsx


In [ ]:
for i in range(len(list_datasets_filtered)):
    list_datasets_filtered[i] = list_datasets_filtered[i].sort_values(by="rt")


In [ ]:
list_datasets_filtered[0]["rt"][0]

np.float64(12.07)

In [ ]:
save_excel(list_datasets_filtered, "../data/datasets_by_config_3_0.xlsx")

Archivo Excel creado: ../data/datasets_by_config_3_0.xlsx


In [ ]:
datasets_sort = list_datasets_filtered


In [38]:
def alpha(rta, rtb, t0):
    return (rta - t0) / (rtb - t0)

def t0(length, flowrate):
    return length/flowrate

def diff(rta, rtb, total):
    return (rtb - rta) / total

VOY A HACER UN BONITO REFACTOR

In [50]:
def main(data, family, fscore):
    
    # Obtiene los tipos de experimentos que hay, y añade una columan experiment para posteriormente agrupar.
    def get_types_of_experiments(data):
        # data['experiment'] = data['id'].apply(lambda x: x.split('_')[0])
        data.loc[:, 'experiment'] = data['id'].apply(lambda x: x.split('_')[0])
        return data['experiment'].unique()

    # Si tiene algo entre paréntesis los datos se lo quita, debido a que en alternative parents están sin paréntesis.
    def get_family_name(family):
        match = re.search(r'\((.*?)\)', family)
        if match:
            family_name = family.split('(')[0].strip()
            return family_name
        return family

    # Obtiene un conjunto de posibles padres (familia) del compuesto, uniendo el nombre de la columna class con alternative_parents.
    def get_posible_families(experiment_data):
        alternative_parents = [set(item.split(",")) for item in experiment_data["alternative_parents"]]
        class_attr = experiment_data["classyfire.class"]
        return [parent_set.union({cls}) for parent_set, cls in zip(alternative_parents, class_attr)]

    # Retorna una lista de datasets separados por experimento, tras haberlos filtrado por la familia.
    def filter_by_family(experiment_data, families, family_name, family):
        return experiment_data[[family_name in conjunto or family in conjunto for conjunto in families]]

    # Filtra el dataset según su configuración 
    def filter_by_config(data_family, config):
        usp_columns = [
            'column.usp.code_0', 'column.usp.code_L1', 'column.usp.code_L10',
            'column.usp.code_L109', 'column.usp.code_L11', 'column.usp.code_L114',
            'column.usp.code_L122', 'column.usp.code_L3', 'column.usp.code_L43',
            'column.usp.code_L68', 'column.usp.code_L7'
        ]
        # Crear una lista de condiciones para las columnas usp_code
        usp_conditions = [
            (data_family[usp_col] == 1) if usp_col == config.columna.usp_code else (data_family[usp_col] == 0)
            for usp_col in usp_columns
        ]

        # Combinar todas las condiciones usando '&'
        combined_usp_condition = usp_conditions[0]
        for condition in usp_conditions[1:]:
            combined_usp_condition &= condition

        # Crear la condición general
        combined_condition = (
            (data_family['column.name'] == config.columna.name) &
            (data_family['column.length'] == config.columna.length) &
            (data_family['column.id'] == config.columna.id) &
            (data_family['column.particle.size'] == config.columna.particle_size) &
            (data_family['column.temperature'] == config.columna.temperature) &
            (data_family['column.flowrate'] == config.columna.flowrate) &
            (data_family['column.t0'] == config.columna.t0) &
            combined_usp_condition  # Condiciones combinadas de usp_code
        )

        # Filtrar los datos que cumplen la condición
        return data_family[combined_condition]

    # Devuelve una lista de objetos de configuración
    def create_config_objects(dataset):
        def get_uspcode(row):
            usp_columns = [
                'column.usp.code_0', 'column.usp.code_L1', 'column.usp.code_L10',
                'column.usp.code_L109', 'column.usp.code_L11', 'column.usp.code_L114',
                'column.usp.code_L122', 'column.usp.code_L3', 'column.usp.code_L43',
                'column.usp.code_L68', 'column.usp.code_L7'
            ]
            # Busca la primera columna con valor 1 y devuelve su nombre
            return next((col for col in usp_columns if row[col] == 1), None)

        config_objects = set()

        for _, row in dataset.iterrows():
            # Crear el objeto Column
            columna = Column(
                name=row['column.name'],
                usp_code=get_uspcode(row),  # Llama a la función para determinar el USP Code
                length=row['column.length'],
                id=row['column.id'],
                particle_size=row['column.particle.size'],
                temperature=row['column.temperature'],
                flowrate=row['column.flowrate'],
                t0=row['column.t0']
            )
            
            # Crear el objeto Config
            
            eluyente1 = next((key for key in dataset.columns if key.startswith('eluent.1.') and row[key] == 100), None)
            eluyente2 = next((key for key in dataset.columns if key.startswith('eluent.2.') and row[key] == 100), None)

            config = Config(eluyente1=eluyente1, eluyente2=eluyente2, columna=columna)
            config_objects.add(config)

        return list(config_objects)



    # Dada los tipos de experimentos separa
    def process_experiments(data, experiments, family_name, family):
        result_datasets = []
        configs = []  # Lista de configuraciones
        for exp in experiments:
            experiment_data = data[data['experiment'] == exp]
            families = get_posible_families(experiment_data)
            data_family = filter_by_family(experiment_data, families, family_name, family)
            
            # Creamos un objeto configuración y lo añadimos a una lista
            exp_configs = create_config_objects(data_family)
            configs.extend(exp_configs)
            
            datasets_by_config = [filter_by_config(data_family, config) for config in exp_configs]
            result_datasets.extend(datasets_by_config)
        
        return result_datasets, configs
    
    def calculate_alpha_results(dataset):
        resultados_dataset = []
        primer_elemento = dataset.iloc[0]
        t0_value = t0(primer_elemento['column.length'], primer_elemento['column.flowrate'])
        
        for i in range(1, len(dataset)):
            elemento_n_menos_1 = dataset.iloc[i - 1]
            elemento_n = dataset.iloc[i]
            resultado = alpha(elemento_n_menos_1['rt'], elemento_n['rt'], t0_value)
            resultados_dataset.append(resultado)
        return resultados_dataset

    def calculate_results(result_datasets):
        resultados = []
        for dataset in result_datasets:
            if dataset.empty:
                continue
            resultados.append(calculate_alpha_results(dataset))
        return resultados

    def build_results_list(configs, resultados, fscore):
        lista_tuplas = []
        for i in range(min(len(configs), len(resultados))):
            # Aplicamos la función fscore a los resultados de cada configuración
            score = fscore(resultados[i])
            lista_tuplas.append((configs[i].__str__(), score.__str__()))  # Emparejamos config con su puntaje
        return lista_tuplas

    # Obtener los tipos de experimentos y el nombre de la familia
    types_of_experiments = get_types_of_experiments(data)
    family_name = get_family_name(family)

    # Procesar experimentos y obtener datasets y configuraciones
    result_datasets, configs = process_experiments(data, types_of_experiments, family_name, family)

    #save_excel(result_datasets, "../data/result_datasets.xlsx")

    # Ordenar datasets por RT
    for dataset in result_datasets:
        dataset.sort_values(by="rt", inplace=True)

    # Calcular resultados de alpha
    resultados = calculate_results(result_datasets)

    # Construir la lista de resultados finales
    lista_tuplas = build_results_list(configs, resultados, fscore)

    return lista_tuplas


In [ ]:
peor_resultado = main(data, "Benzene and substituted derivatives (CHEMONTID:0002279)", fscore=fscore_peor_caso)
medio_resultado = main(data, "Benzene and substituted derivatives (CHEMONTID:0002279)", fscore=fscore_caso_medio)
mejor_resultado = main(data, "Benzene and substituted derivatives (CHEMONTID:0002279)", fscore=fscore_mejor_caso)

Archivo Excel creado: ../data/result_datasets.xlsx
Archivo Excel creado: ../data/result_datasets.xlsx
Archivo Excel creado: ../data/result_datasets.xlsx


In [ ]:
print(len(peor_resultado))
print(len(medio_resultado))
print(len(peor_resultado))

275
275
275


In [ ]:
peor_segundos_terminos = [tupla[1] for tupla in peor_resultado]
medio_segundos_terminos = [tupla[1] for tupla in medio_resultado]
mejor_segundos_terminos = [tupla[1] for tupla in mejor_resultado]
print(peor_segundos_terminos)
print(medio_segundos_terminos)
print(mejor_segundos_terminos)

['1.0000378827852505', '1.0003161222339305', '1.0008168698016526', '1.0', '1.0000495011244566', '1.0018756698821008', '1.0', '0', '1.000133864335197', '1.0443768340398019', '0', '1.0', '1.0003920334462861', '1.0059411112314995', '1.0016278462187744', '1.0000905760637657', '1.0010214504596529', '1.0049287360245798', '1.000303137472844', '1.001414570071739', '1.0019344400675296', '1.0000156057429135', '1.0000405145852507', '1.0041476565740357', '1.0001619782949085', '1.0', '1.0062892018461849', '1.000138056713698', '1.0000025542372017', '1.0', '1.0', '1.0', '1.0', '1.000035266526776', '1.000141303952877', '1.000412065271139', '1.0000243886576483', '1.0', '1.0030370857119388', '1.0000072085372556', '1.0002161782797487', '1.0', '1.0', '1.0000470642022954', '1.000047035424394', '1.0000173977601479', '0', '1.017045922994476', '1.00080685015784', '1.000465620033808', '1.001110583864226', '1.0170462167098324', '0', '1.0164027382572476', '0', '1.0107099743084431', '1.0083003912608304', '1.00958

In [ ]:
import csv
with open("../data/scoring.csv", mode='w', newline='') as file:
        writer = csv.writer(file)
        
        # Escribir los encabezados del CSV
        writer.writerow(['Config', 'Score'])
        
        # Escribir cada tupla (config, score) en una fila
        for config, score in resultado:
            writer.writerow([repr(config), score])


In [41]:
def main_2(data, family, fscore):
    
    # Obtiene los tipos de experimentos que hay, y añade una columan experiment para posteriormente agrupar.
    def get_types_of_experiments(data):
        # data['experiment'] = data['id'].apply(lambda x: x.split('_')[0])
        data.loc[:, 'experiment'] = data['id'].apply(lambda x: x.split('_')[0])
        return data['experiment'].unique()

    # Si tiene algo entre paréntesis los datos se lo quita, debido a que en alternative parents están sin paréntesis.
    def get_family_name(family):
        match = re.search(r'\((.*?)\)', family)
        if match:
            family_name = family.split('(')[0].strip()
            return family_name
        return family

    # Obtiene un conjunto de posibles padres (familia) del compuesto, uniendo el nombre de la columna class con alternative_parents.
    def get_posible_families(experiment_data):
        alternative_parents = [set(item.split(",")) for item in experiment_data["alternative_parents"]]
        class_attr = experiment_data["classyfire.class"]
        return [parent_set.union({cls}) for parent_set, cls in zip(alternative_parents, class_attr)]

    # Retorna una lista de datasets separados por experimento, tras haberlos filtrado por la familia.
    def filter_by_family(experiment_data, families, family_name, family):
        return experiment_data[[family_name in conjunto or family in conjunto for conjunto in families]]

    # Filtra el dataset según su configuración 
    def filter_by_config(data_family, config):
        usp_columns = [
            'column.usp.code_0', 'column.usp.code_L1', 'column.usp.code_L10',
            'column.usp.code_L109', 'column.usp.code_L11', 'column.usp.code_L114',
            'column.usp.code_L122', 'column.usp.code_L3', 'column.usp.code_L43',
            'column.usp.code_L68', 'column.usp.code_L7'
        ]
        # Crear una lista de condiciones para las columnas usp_code
        usp_conditions = [
            (data_family[usp_col] == 1) if usp_col == config.columna.usp_code else (data_family[usp_col] == 0)
            for usp_col in usp_columns
        ]

        # Combinar todas las condiciones usando '&'
        combined_usp_condition = usp_conditions[0]
        for condition in usp_conditions[1:]:
            combined_usp_condition &= condition

        # Crear la condición general
        combined_condition = (
            (data_family['column.name'] == config.columna.name) &
            (data_family['column.length'] == config.columna.length) &
            (data_family['column.id'] == config.columna.id) &
            (data_family['column.particle.size'] == config.columna.particle_size) &
            (data_family['column.temperature'] == config.columna.temperature) &
            (data_family['column.flowrate'] == config.columna.flowrate) &
            (data_family['column.t0'] == config.columna.t0) &
            combined_usp_condition  # Condiciones combinadas de usp_code
        )

        # Filtrar los datos que cumplen la condición
        return data_family[combined_condition]

    # Devuelve una lista de objetos de configuración
    def create_config_objects(dataset):
        def get_uspcode(row):
            usp_columns = [
                'column.usp.code_0', 'column.usp.code_L1', 'column.usp.code_L10',
                'column.usp.code_L109', 'column.usp.code_L11', 'column.usp.code_L114',
                'column.usp.code_L122', 'column.usp.code_L3', 'column.usp.code_L43',
                'column.usp.code_L68', 'column.usp.code_L7'
            ]
            # Busca la primera columna con valor 1 y devuelve su nombre
            return next((col for col in usp_columns if row[col] == 1), None)

        config_objects = set()

        for _, row in dataset.iterrows():
            # Crear el objeto Column
            columna = Column(
                name=row['column.name'],
                usp_code=get_uspcode(row),  # Llama a la función para determinar el USP Code
                length=row['column.length'],
                id=row['column.id'],
                particle_size=row['column.particle.size'],
                temperature=row['column.temperature'],
                flowrate=row['column.flowrate'],
                t0=row['column.t0']
            )
            
            # Crear el objeto Config
            
            eluyente1 = next((key for key in dataset.columns if key.startswith('eluent.1.') and row[key] == 100), None)
            eluyente2 = next((key for key in dataset.columns if key.startswith('eluent.2.') and row[key] == 100), None)

            config = Config(eluyente1=eluyente1, eluyente2=eluyente2, columna=columna)
            config_objects.add(config)

        return list(config_objects)



    # Dada los tipos de experimentos separa
    def process_experiments(data, experiments, family_name, family):
        result_datasets = []
        configs = []  # Lista de configuraciones
        for exp in experiments:
            experiment_data = data[data['experiment'] == exp]
            families = get_posible_families(experiment_data)
            data_family = filter_by_family(experiment_data, families, family_name, family)
            
            # Creamos un objeto configuración y lo añadimos a una lista
            exp_configs = create_config_objects(data_family)
            configs.extend(exp_configs)
            
            datasets_by_config = [filter_by_config(data_family, config) for config in exp_configs]
            result_datasets.extend(datasets_by_config)
        
        return result_datasets, configs
    
    def calculate_alpha_results(dataset):
        resultados_dataset = []
        ultimo_elemento = dataset.iloc[len(dataset)-1]
        total = ultimo_elemento['rt']
        
        for i in range(1, len(dataset)):
            elemento_n_menos_1 = dataset.iloc[i - 1]
            elemento_n = dataset.iloc[i]
            resultado = diff(elemento_n_menos_1['rt'], elemento_n['rt'], total)
            resultados_dataset.append(resultado)
        return resultados_dataset

    def calculate_results(result_datasets):
        resultados = []
        for dataset in result_datasets:
            if dataset.empty:
                continue
            resultados.append(calculate_alpha_results(dataset))
        return resultados

    def build_results_list(configs, resultados, fscore):
        lista_tuplas = []
        for i in range(min(len(configs), len(resultados))):
            # Aplicamos la función fscore a los resultados de cada configuración
            score = fscore(resultados[i])
            lista_tuplas.append((configs[i].__str__(), score.__str__()))  # Emparejamos config con su puntaje
        return lista_tuplas

    # Obtener los tipos de experimentos y el nombre de la familia
    types_of_experiments = get_types_of_experiments(data)
    family_name = get_family_name(family)

    # Procesar experimentos y obtener datasets y configuraciones
    result_datasets, configs = process_experiments(data, types_of_experiments, family_name, family)

    #save_excel(result_datasets, "../data/result_datasets.xlsx")

    # Ordenar datasets por RT
    for dataset in result_datasets:
        dataset.sort_values(by="rt", inplace=True)

    # Calcular resultados de alpha
    resultados = calculate_results(result_datasets)

    # Construir la lista de resultados finales
    lista_tuplas = build_results_list(configs, resultados, fscore)

    return lista_tuplas


In [42]:
peor_resultado = main_2(data, "Benzene and substituted derivatives (CHEMONTID:0002279)", fscore=fscore_peor_caso)
medio_resultado = main_2(data, "Benzene and substituted derivatives (CHEMONTID:0002279)", fscore=fscore_caso_medio)
mejor_resultado = main_2(data, "Benzene and substituted derivatives (CHEMONTID:0002279)", fscore=fscore_mejor_caso)

In [40]:
peor_segundos_terminos = [tupla[1] for tupla in peor_resultado]
medio_segundos_terminos = [tupla[1] for tupla in medio_resultado]
mejor_segundos_terminos = [tupla[1] for tupla in mejor_resultado]
print(peor_segundos_terminos)
print(medio_segundos_terminos)
print(mejor_segundos_terminos)

['0.0010335917312661277', '0.002298850574712676', '0.05776379100123613', '0.0', '0.0017617801104799157', '0.38888888888888884', '0.0', '0', '0.005582693929599807', '0.5535593982235095', '0', '0.0', '0.016252390057361368', '0.16735452298591796', '0.3360995850622407', '0.0037037037037037346', '0.032473837522925895', '0.06438127090300999', '0.02898550724637684', '0.040756914119359534', '0.022088353413654577', '0.018633540372670825', '0.11250000000000002', '0.06451612903225806', '0.00043535045711797074', '0.0', '0.06078431372549021', '0.0038971161340608505', '0.00019114554998450763', '0.0', '0.0', '0.0', '0.0', '0.0007698229407236257', '0.006515264333581561', '0.009505703422053199', '0.0007467330429371783', '0.0', '0.0626035955629223', '0.001181012632163657', '0.01488551684184499', '0.0', '0.0', '0.003058103975535171', '0.0030303030303030333', '0.0021306372585300935', '0', '0.557277628032345', '0.03136025088200705', '0.016874541452677947', '0.0881410256410256', '0.5351473922902495', '0', '

In [45]:

peor_resultado = main_2(data, "Organooxygen compounds (CHEMONTID:0000323)", fscore=fscore_peor_caso)
medio_resultado = main_2(data, "Organooxygen compounds (CHEMONTID:0000323)", fscore=fscore_caso_medio)
mejor_resultado = main_2(data, "Organooxygen compounds (CHEMONTID:0000323)", fscore=fscore_mejor_caso)

In [46]:
peor_segundos_terminos = [tupla[1] for tupla in peor_resultado]
medio_segundos_terminos = [tupla[1] for tupla in medio_resultado]
mejor_segundos_terminos = [tupla[1] for tupla in mejor_resultado]
print(peor_segundos_terminos)
print(medio_segundos_terminos)
print(mejor_segundos_terminos)

['0.006042296072507558', '0.0', '0.06250000000000006', '0.0', '0.002503661594395906', '0.0013612122990968062', '0.006060606060606039', '0.0', '0.00011725163746807784', '0.0', '0.0014825796886583655', '0', '0.23752969121140144', '0.0024937655860348875', '0.0003814973772054897', '0.05131298134916961', '0.9471285323609845', '0.7016574585635359', '0', '0.7634146341463415', '0.2156804733727811', '0.015706806282722457', '0.3340122199592668', '0', '0.004626247869491122', '0.8567375886524823', '0.004277563670196437', '0.00573613766730402', '0.0', '0.0038216560509554457', '0.0026539278131634783', '0.0', '0.03978494623655917', '0.02369077306733168', '0.0061117578579744135', '0.0025929127052722388', '0', '0.0035639286871958402', '0.0018150286917499115', '0.0', '0.010721396844846081', '0.002923976608187202', '0.008746355685131267', '0.013926747142497286', '0', '0.7257683215130024', '0.12328767123287664', '0.0071465033180194044', '0.0', '0', '0', '0.0', '0.027777777777777714', '0.0', '0', '0', '0',

In [51]:
peor_resultado = main(data, "Organooxygen compounds (CHEMONTID:0000323)", fscore=fscore_peor_caso)
medio_resultado = main(data, "Organooxygen compounds (CHEMONTID:0000323)", fscore=fscore_caso_medio)
mejor_resultado = main(data, "Organooxygen compounds (CHEMONTID:0000323)", fscore=fscore_mejor_caso)

In [52]:
peor_segundos_terminos = [tupla[1] for tupla in peor_resultado]
medio_segundos_terminos = [tupla[1] for tupla in medio_resultado]
mejor_segundos_terminos = [tupla[1] for tupla in mejor_resultado]
print(peor_segundos_terminos)
print(medio_segundos_terminos)
print(mejor_segundos_terminos)

['1.0002246626049094', '1.0', '1.0007853002592166', '1.0', '1.000063022601659', '1.000004597270261', '1.0000670825786542', '1.0', '1.0000036909048269', '1.0', '1.000046380417642', '0', '1.0020169829968333', '1.0000300622288136', '1.0000122414253916', '1.0019008662175168', '1.0667323213941637', '1.006407992330592', '0', '1.032638164754953', '1.0001458986274723', '1.0012717253073335', '1.0947307618253002', '0', '1.000230126367637', '1.016415273814377', '1.0000625386042836', '1.000120510966498', '1.0', '1.0001849036446564', '1.0000884815632365', '1.0', '1.0006002060166597', '1.0007632054629443', '1.0001688415060663', '1.0001205448627797', '0', '1.0000172383126422', '1.0000276486506539', '1.0', '1.000568430968119', '1.0000471785539773', '1.0001414689189416', '1.0001225434209502', '0', '1.003083041264549', '1.0004516485170873', '1.0001403396218849', '1.0', '0', '0', '1.0', '1.000070176845651', '1.0', '0', '0', '0', '0', '0', '0', '0', '1.002568493150685', '0', '0', '1.0000802342841095', '0'